In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import geopandas as gpd


In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Data Cleaning")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "9g") 
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/09 15:32:01 WARN Utils: Your hostname, DESKTOP-702MS12 resolves to a loopback address: 127.0.1.1; using 172.17.210.38 instead (on interface eth0)
24/09/09 15:32:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/09 15:32:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 39170)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/kallto/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/home/kallto/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/home/kallto/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 271, in accum_updates


# Join merchant & consumer data

In [3]:
def join_func(left: DataFrame, right: DataFrame, left_keys: ArrayType, 
              method: StringType='inner', right_keys: ArrayType=None, drop_left: BooleanType=False) -> DataFrame:
    """
    Joins two pyspark dataframes on the left and right specified keys, returning the inner join by default.
    Identifies records not joined in left and right datasets.
    """
    
    if not right_keys:
        right_keys = left_keys

    join_cond = "&".join([f"(left.{i} == right.{j})" for i,j in zip(left_keys, right_keys)])

    if not drop_left:
        drop_cols = [right[c] for c in right_keys]
    else: 
        drop_cols = [left[c] for c in left_keys]

    # Join
    joined_df = left.join(right, eval(join_cond), how=method).drop(*drop_cols)

    # See which return all the rows from left df that do not have a match in the right df
    mismatch = left.join(right, eval(join_cond), how='left_anti').drop(*drop_cols)
    print(f"Number of unjoined left records: {mismatch.count()}")

    return joined_df

In [4]:
# Load in merchant data (parquet)
merchant = spark.read.parquet("../data/tables/part_1/tbl_merchants.parquet")

# Load in merchant fraud (csv)
merchant_fp = pd.read_csv("../data/tables/part_1/merchant_fraud_probability.csv")
merchant_fp = spark.createDataFrame(merchant_fp)

# Load in consumer list (csv)
consumer = pd.read_csv("../data/tables/part_1/tbl_consumer.csv", delimiter="|")
consumer = spark.createDataFrame(consumer)

# Load in consumer fraud (csv)
consumer_fp = pd.read_csv("../data/tables/part_1/consumer_fraud_probability.csv")
consumer_fp = spark.createDataFrame(consumer_fp)

consumer_ud = spark.read.parquet("../data/tables/part_1/consumer_user_details.parquet")

**Join merchants data**

`tbl_merchant` to `merchant_fraud_probability`

In [5]:
merchant_full = join_func(merchant, merchant_fp, ['merchant_abn'], 'left', drop_left=False)
merchant_full.orderBy('fraud_probability', ascending=True).show(5)

Number of unjoined left records: 3978
+--------------------+--------------------+------------+--------------+-----------------+
|                name|                tags|merchant_abn|order_datetime|fraud_probability|
+--------------------+--------------------+------------+--------------+-----------------+
|        Mollis Corp.|((watch, clock, a...| 12516851436|          NULL|             NULL|
|     Ante Industries|[[motor vehicle s...| 15613631617|          NULL|             NULL|
|Pellentesque Habi...|([cable, Satellit...| 19839532017|          NULL|             NULL|
|Mauris Nulla Inte...|((opticians, opti...| 34440496342|          NULL|             NULL|
|Quis Tristique Ac...|[(watch, clock, a...| 35344855546|          NULL|             NULL|
+--------------------+--------------------+------------+--------------+-----------------+
only showing top 5 rows



**Join customer data**

`tbl_consumer` to `consumer_user_detail` and `consumer_fraud_probability`

In [6]:
consumer_full = join_func(consumer, consumer_ud, ['consumer_id'], 'left') # match user_id to consumers
consumer_full = join_func(consumer_full, consumer_fp, ['user_id'], 'left') # match fraud probaibility to consumers

# Drop irrelevant columns
consumer_full = consumer_full.drop(*['address', 'order_datetime'])
consumer_full.show(10)

24/09/09 15:32:40 WARN TaskSetManager: Stage 18 contains a task of very large size (3465 KiB). The maximum recommended task size is 1000 KiB.


Number of unjoined left records: 0


24/09/09 15:32:42 WARN TaskSetManager: Stage 25 contains a task of very large size (3465 KiB). The maximum recommended task size is 1000 KiB.


Number of unjoined left records: 479871


24/09/09 15:32:44 WARN TaskSetManager: Stage 32 contains a task of very large size (3465 KiB). The maximum recommended task size is 1000 KiB.


+-----------------+-----+--------+------+-----------+-------+------------------+
|             name|state|postcode|gender|consumer_id|user_id| fraud_probability|
+-----------------+-----+--------+------+-----------+-------+------------------+
| Yolanda Williams|   WA|    6935|Female|    1195503|      1|  9.80543113652096|
|       Mary Smith|  NSW|    2782|Female|     179208|      2| 9.599513915425788|
|       Mary Smith|  NSW|    2782|Female|     179208|      2|10.069850934775245|
|    Jill Jones MD|   NT|     862|Female|    1194530|      3| 8.300636455314633|
|  Lindsay Jimenez|  NSW|    2780|Female|     154128|      4|  9.63330241109042|
|Rebecca Blanchard|   WA|    6355|Female|     712975|      5|  9.02022421158597|
|Rebecca Blanchard|   WA|    6355|Female|     712975|      5|10.868364868449886|
|Rebecca Blanchard|   WA|    6355|Female|     712975|      5|27.496186536467164|
|    Karen Chapman|  NSW|    2033|Female|     407340|      6|10.459280127078758|
|     Andrea Jones|  QLD|   

**Join customers and transaction data**

In [7]:
# Read transaction dataset
transaction1 = spark.read.parquet("../data/tables/part_2")
transaction2 = spark.read.parquet("../data/tables/part_3")
transaction3 = spark.read.parquet("../data/tables/part_4")

transaction = transaction1.union(transaction2).union(transaction3)

In [18]:
# Join customers to transactions
consumer_transaction = consumer_full.join(transaction, on='user_id')

# Count number of customers in dataset with no transactions
mismatch_c_t = consumer_full.join(transaction, on='user_id', how='left_anti')
print(f"There are {mismatch_c_t.count()} customers with no matching transactions")

24/09/09 15:39:57 WARN TaskSetManager: Stage 128 contains a task of very large size (3465 KiB). The maximum recommended task size is 1000 KiB.


There are 475918 customers with no matching transactions


In [40]:
consumer_full.select(['postcode', 'gender', 'user_id', 'fraud_probability']).join(
)
transaction.select(['user_id', 'merchant_abn'])

TypeError: DataFrame.join() missing 1 required positional argument: 'other'

24/09/09 17:38:08 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 6177438 ms exceeds timeout 120000 ms
24/09/09 17:38:10 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/09 17:38:10 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [22]:
consumer_transaction.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- consumer_id: long (nullable = true)
 |-- fraud_probability: double (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



# Join external datasets

In [ ]:
# Load in dataset
medians = pd.read_csv("../data/tables/sa2_dataset/main/C21_G02_SA2_filtered.csv")

# Column names
variables = {'REGION': 'sa2_code',
             1: "median_age", 
             2: "median_total_personal_income",
             3: "median_total_family_income",
             4: "median_total_household_income",
             5: "median_mortgage_repayment",
             6: "median_rent",
             7: "avg_people_per_bedroom",
             8: "avg_household_size"}

medians = medians.pivot(index='REGION', columns=['MEDAVG'], values='OBS_VALUE').reset_index().rename(columns=variables)
medians.columns.name = None
medians['sa2_code'] = medians.sa2_code.astype(str)

medians.head(5)

In [ ]:
# Read in zone file to identify SA2 zones with null median/average values
sa2_boundary_gdf = gpd.read_file("../data/tables/sa2_boundary/SA2_2021_AUST_GDA2020.shp")
sa2_names = sa2_boundary_gdf[['SA2_CODE21', 'SA2_NAME21']].astype('string')

# Find records with null statistics
null_regions = medians[medians.isna().any(axis=1)]
null_regions = null_regions.merge(sa2_names, left_on='sa2_code', right_on='SA2_CODE21')
null_regions.iloc[:,-2:]